In [66]:
from dotenv import load_dotenv
import openai
import os

In [67]:
from openai import OpenAI

In [68]:
# Load environment variables from the .env file
load_dotenv()

True

In [69]:
# Access the variables
api_key = os.getenv('OPENAI_API_KEY')

In [70]:
client = openai.OpenAI(api_key=api_key)

## Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

In [1]:
!curl localhost:9200

{
  "name" : "ae8924fd33b9",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "lFqQwODFSie7XvguP6iT6A",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


> Answer: `"build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73"`

## Getting the data

Now let's get the FAQ data. You can run this snippet:

```python
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Note that you need to have the `requests` library:

```bash
pip install requests
```

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

In [5]:
from elasticsearch import Elasticsearch

In [6]:
es_client=Elasticsearch('http://localhost:9200')

In [7]:
index_settings={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name='homework-questions'

es_client.indices.create(index=index_name,body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'homework-questions'})

In [8]:
for doc in documents:
    es_client.index(index=index_name,document=doc)

> Answer: `index`

## Q3. Searching

Now let's search in our index. 

We will execute a query "How do I execute a command in a running docker container?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 94.05
* 84.05
* 74.05
* 64.05

Look at the `_score` field.

In [10]:
query='How do I execute a command in a running docker container?'

In [11]:
def elastic_search(query):
    search_query={
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
    }
    response=es_client.search(index=index_name, body=search_query)

    top_score = response['hits']['hits'][0]['_score']
        
    return top_score

In [12]:
elastic_search(query)

84.050095

> Answer: 84.05

## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [60]:
def elastic_search(query):
    search_query={
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
    }
    response=es_client.search(index=index_name, body=search_query)
    
    result_docs=[]
    
    for response in response['hits']['hits']:
        result_docs.append(response['_source'])
        
    return result_docs

In [19]:
elastic_search(query)[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 962
* 1462
* 1962
* 2462

In [41]:
records=elastic_search(query)

In [42]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

context = "\n\n".join([context_template.format(question=record['question'], text=record['text']) for record in records])

question = "How do I execute a command in a running docker container?"

prompt = prompt_template.format(question=question, context=context)

prompt_length = len(prompt)
print(prompt_length)


1462


> Answer: 1462

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

In [43]:
import tiktoken

# Initialize the encoding for GPT-4 model
encoding = tiktoken.encoding_for_model("gpt-4")

# Encode the prompt and count the number of tokens
tokens = encoding.encode(prompt)
num_tokens = len(tokens)

print(num_tokens)


325


> Answer: 322

## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.

In [81]:
def build_prompt(query,search_results):
    
    context_template = """
    Q: {question}
    A: {text}
    """.strip()
    
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    ccontext = "\n\n".join([context_template.format(question=record['question'], text=record['text']) for record in records])
    
    for doc in search_results:
        context=context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        
    prompt=prompt_template.format(question=query,context=context).strip()
    return prompt

In [82]:
def llm(prompt):
    response=client.chat.completions.create(model='gpt-4o', messages=[{"role":"user","content":prompt}])
    return response.choices[0].message.content

In [83]:
def rag(query):
    search_results=elastic_search(query)
    prompt=build_prompt(query,search_results)
    answer=llm(prompt)
    return answer

In [85]:
answer=llm(prompt)

In [86]:
answer

'To execute a command in a running Docker container, follow these steps:\n\n1. Use the `docker ps` command to find the container ID of the running container:\n    ```sh\n    docker ps\n    ```\n\n2. Execute a command in the specific container using `docker exec`. For example, to start a bash shell, use:\n    ```sh\n    docker exec -it <container-id> bash\n    ```\n\nReplace `<container-id>` with the actual ID of the running container.'

## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.

In [87]:
# Constants
tokens_per_request_input = 150
tokens_per_request_output = 250
cost_per_1k_input_tokens = 0.005
cost_per_1k_output_tokens = 0.015
number_of_requests = 1000

# Calculate total tokens
total_input_tokens = tokens_per_request_input * number_of_requests
total_output_tokens = tokens_per_request_output * number_of_requests

# Convert tokens to thousands of tokens
total_input_tokens_in_k = total_input_tokens / 1000
total_output_tokens_in_k = total_output_tokens / 1000

# Calculate cost
cost_input = total_input_tokens_in_k * cost_per_1k_input_tokens
cost_output = total_output_tokens_in_k * cost_per_1k_output_tokens

# Total cost
total_cost = cost_input + cost_output

print(f"Total cost to run {number_of_requests} requests: ${total_cost:.2f}")


Total cost to run 1000 requests: $4.50
